In [50]:
# Reading of the datasets and understanding the size,the no. of null values and various other elements

import pandas as pd 

gt =  pd.read_csv("datasets\Genotype_Data.csv")
pt=pd.read_csv("datasets\Phenotypic_Data.csv")

pt.size
gt.size
pt.head()
gt.head()

gt.info()
pt.info()
gt.isnull().sum()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14790 entries, 0 to 14789
Columns: 291 entries, SNPs to WH1284
dtypes: float64(3), int64(1), object(287)
memory usage: 32.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 281 entries, 0 to 280
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Genotype             280 non-null    object 
 1   DH_Dharwad           280 non-null    float64
 2   DH_IARI-DELHI        280 non-null    float64
 3   DH_IARI-Jharkhand    280 non-null    float64
 4   DH_KARNAL            280 non-null    float64
 5   DH_Pooled            281 non-null    float64
 6   GFD_Dharwad          280 non-null    float64
 7   GFD_IARI-Delhi       280 non-null    float64
 8   GFD_IARI-Jharkhand   280 non-null    float64
 9   GFD_Karnal           280 non-null    float64
 10  GFD_Pooled           281 non-null    float64
 11  GNPS_Dharwad         280 non-null    float64
 12  GNPS_IARI-Jh

SNPs          0
alleles       0
Chrom         0
Pos           0
strand        0
           ... 
WH1279      392
WH1280      165
WH1281     1303
WH1283      508
WH1284     1077
Length: 291, dtype: int64

In [51]:
# Removing the useless or no related cols from the dataset
col=pt['Genotype']
cols_to_drop=['Chrom','Pos','strand','assembly','center','protLSID','assayLSID','panel','QCcode']
gt=gt.drop(cols_to_drop,axis=1)
gt.to_csv("updated_genotype",index=False)


In [52]:
gt.head()

,SNPs,alleles,AAI-W22,AAI-W29,AKAW5080,AKAW5088,AKAW5099,BCW5,BRW3863,BRW3869,...,WH1274,WH1275,WH1276,WH1277,WH1278,WH1279,WH1280,WH1281,WH1283,WH1284
0,AX-94381285,C/T,TC,CC,CC,CC,CC,CC,TC,CC,...,CC,CC,CC,CC,CC,CC,CC,CC,NaN,CC
1,AX-94383718,A/G,AA,AA,AA,AA,AA,AA,AG,AA,...,AA,AA,AA,AA,AA,AA,AA,AA,AA,AA
2,AX-94384181,A/G,AA,NaN,NaN,AA,AA,GG,NaN,GG,...,GG,NaN,NaN,NaN,AA,GG,GG,AA,GG,AA
3,AX-94384966,T/C,CC,TT,CC,CC,TT,TT,CC,TT,...,CC,TT,CC,TT,TT,TT,TT,NaN,CC,CC
4,AX-94386458,C/A,CC,AA,AC,CC,CC,CC,AA,CC,...,CC,CC,CC,CC,CC,CC,AA,NaN,CC,NaN


In [53]:
df= pd.read_csv("updated_genotype")
df.isnull().sum()


SNPs           0
alleles        0
AAI-W22      395
AAI-W29     1155
AKAW5080     624
            ... 
WH1279       392
WH1280       165
WH1281      1303
WH1283       508
WH1284      1077
Length: 282, dtype: int64

In [54]:


# Replacing the null values for given dataset with primary allele

import pandas as pd

def replace_nan_with_first_letter(value, alleles):
    if pd.isna(value):
        letter_before_slash = alleles.split('/')[0]
        return letter_before_slash * 2
    else:
        return value

for column in df.columns[1:]:
    df[column] = df.apply(lambda row: replace_nan_with_first_letter(row[column], row['alleles']), axis=1)

df=df.drop(['alleles'],axis=1)
df.to_csv("final_genotype",index=False)

In [55]:
# Removing the other cols from phenotype dataset as we consider only pooled values

dropme=[]
for i in pt.columns:
    if i=='DH_Pooled' or i=='GFD_Pooled' or i=='GNPS_Pooled' or i=='GWPS_Pooled' or i=='PH_Pooled' or i=='GY_Pooled':
        continue
    else:
        dropme.append(i)
dropme=dropme[1:]

pt=pt.drop(dropme,axis=1)
pt.to_csv("final_phenotype",index=False)

In [56]:
# Label Encoding and replacing the encoded values

geno_map={'AA':1,'AT':2,'AG':3,'AC':4,'TT':5,'TG':6,'TC':7,'GG':8,'CG':9,'CC':10}
sample_geno=pd.read_csv("final_genotype")
df2=pd.DataFrame(sample_geno)

for i in df2[1:]:
    df2[i]=df2[i].map(geno_map)

df2['SNPs']=df['SNPs']
df2.head()
df2.to_csv("fgenotype",index=False)


In [57]:
# import pandas as pd 

# seq=""
# snps=[]
# coms=[]
# cm=pd.read_csv("final_genotype")
# for i in cm.values:
#     for j in i:
#         snps.append(j)
#         i=i[1:]
#         seq+=j
#     if(seq not in coms):
#         coms.append(seq)
#     else:
#         snps.remove(snps[len(snps)-1])
# print(len(snps))
    

    


In [58]:
# Merging both the dataset based on genotype  

import pandas as pd

fg=pd.read_csv("fgenotype")
fp=pd.read_csv("final_phenotype")



fg = fg.set_index('SNPs').T.reset_index()

fg.columns.name = None
fg = fg.rename(columns={'index': 'Genotype'})

merged_dataset = pd.merge(fg, fp, on='Genotype')

merged_dataset.to_csv("mergeds",index=False)



In [59]:
# Finding the Coorelation among the snp data and traits(pooled values) and Finding the number of common snps which highly influence the majority data
p=[]
f=[]
import pandas as pd
poled=[ 'DH_Pooled','GFD_Pooled' ,'GNPS_Pooled','GWPS_Pooled','PH_Pooled','GY_Pooled']

md=pd.read_csv("mergeds")

snp_data = md.iloc[:, 1:14790]

for i in poled:
    gfd_pooled = md[i]
    corr_gfd= snp_data.corrwith(gfd_pooled)
    snp_corr_df = pd.DataFrame({'SNP': snp_data.columns, 'Correlation': corr_gfd})
    snp_corr_df = snp_corr_df.sort_values(by='Correlation', ascending=False)

    k=[]

    for i ,j in  enumerate(snp_corr_df['Correlation']):
        if(j>0):
            k.append(snp_corr_df['SNP'].iloc[i])
    k = k[0:len(k)*4//5]
    p.append(k)


for i in p:
    print(len(i))
f = p
list1 = f[0]
list2 = f[1]
list3 = f[2]
list4 = f[3]
list5 = f[4]
list6 = f[5]




set1 = set(list1)
set2 = set(list2)
set3 = set(list3)
set4 = set(list4)
set5 = set(list5)
set6 = set(list6)
# Find the common elements
common_elements = set1.intersection(set2,set3,set4,set5,set6)
#common_elements = set(common_elements).intersection(set3)

# Convert the result back to a list (if needed)
common_elements_list = list(common_elements)

print(len(common_elements_list))
    



5878
6084
5863
5925
6052
5869
24


In [60]:
# Removing the non related snps from the dataset
dp=[]
for i in sorted(md.columns[1:]):
    if i not in common_elements_list:
        dp.append(i)

poled=[ 'DH_Pooled','GFD_Pooled' ,'GNPS_Pooled','GWPS_Pooled','PH_Pooled','GY_Pooled']
for i in poled:
    dp.remove(i)


In [61]:
# Reading the Final dataset

md1=pd.read_csv("mergeds")
md1=md1.drop(columns=dp)
md1.to_csv("pheno_geno",index=False)


In [62]:
len(md1['Genotype'])

280

In [63]:
# # Building the DL model ->(FNN model) and Predicting the values

# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.metrics import mean_squared_error

# # Load your dataset
# # Assuming df contains your data
# df = md1

# # X should be the genetic sequences
# # y should be the trait values (DH_Pooled, GFD_Pooled, GNPS_Pooled, GWPS_Pooled, PH_Pooled, GY_Pooled)
# X = df.iloc[:, 1:25]  # Assuming the DNA sequences start from the second column

# # Extract the trait values
# y = df[['DH_Pooled', 'GFD_Pooled', 'GNPS_Pooled', 'GWPS_Pooled', 'PH_Pooled', 'GY_Pooled']]

# # Split the data into training and test sets
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # Standardize input features and trait values separately
# scaler_X = MinMaxScaler()
# scaler_y = MinMaxScaler()

# X_train_scaled = scaler_X.fit_transform(X_train)
# X_test_scaled = scaler_X.transform(X_test)
# y_train_scaled = scaler_y.fit_transform(y_train)
# y_test_scaled = scaler_y.transform(y_test)

# # Build the FNN model
# model = keras.Sequential([
#     keras.layers.Input(shape=(X_train.shape[1],)),
#     keras.layers.Dense(128, activation='elu'),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(32, activation='elu'),
#     keras.layers.Dense(6, activation='sigmoid')  # Use sigmoid activation for output layer for values between 0 and 1
# ])

# # Compile the model
# model.compile(optimizer='adam', loss='mean_squared_error')

# # Train the model
# model.fit(X_train_scaled, y_train_scaled, epochs=50, batch_size=10, validation_split=0.2)

# # Evaluate the model on the test set
# y_pred_scaled = model.predict(X_test_scaled)
# mae = mean_squared_error(y_test_scaled, y_pred_scaled)

# # Generate and predict a new sequence
# new_sequence_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,1,2,3,4,4]).reshape(1, -1)  # Reshape to match input shape
# scaled_new_sequence = scaler_X.transform(new_sequence_1)
# predictions_scaled = model.predict(scaled_new_sequence)

# # Inverse transform the scaled predictions to get the original scale
# predictions = scaler_y.inverse_transform(predictions_scaled)

# print("Root Mean Squared Error (RMSE):", np.sqrt(mae))
# print("Predictions:", predictions)


In [68]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import *

# Load your dataset
# Assuming df contains your data
df = md1

# X should be the genetic sequences
# y should be the trait values (DH_Pooled, GFD_Pooled, GNPS_Pooled, GWPS_Pooled, PH_Pooled, GY_Pooled)
X = df.iloc[:, 1:25]  # Assuming the DNA sequences start from the second column

# Extract the trait values
y = df[['DH_Pooled', 'GFD_Pooled', 'GNPS_Pooled', 'GWPS_Pooled', 'PH_Pooled', 'GY_Pooled']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize input features and trait values separately (using the same scalers)
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

# Build and train a Random Forest Regressor model
random_forest_model = RandomForestRegressor(n_estimators=100, random_state=42)  # You can adjust hyperparameters
random_forest_model.fit(X_train_scaled, y_train_scaled)

# Predict on the test set
y_pred_scaled = random_forest_model.predict(X_test_scaled)

# Calculate Mean Squared Error
mae = mean_squared_error(y_test_scaled, y_pred_scaled)

# Generate and predict a new sequence
new_sequence_1 = np.array([1,2,3,4,5,6,7,8,9,10,1,2,3,4,5,6,7,8,9,1,2,3,4,4]).reshape(1, -1)  # Reshape to match input shape
scaled_new_sequence = scaler_X.transform(new_sequence_1)
predictions_scaled = random_forest_model.predict(scaled_new_sequence)

# Inverse transform the scaled predictions to get the original scale
predictions = scaler_y.inverse_transform(predictions_scaled)


print("Root Mean Squared Error (MSE):", np.sqrt(mae))
print("Predictions:", predictions)





Root Mean Squared Error (MSE): 0.12774364993275047
Predictions: [[ 83.92294117  42.7794525   48.03183817   2.23483025  95.35428258
  425.35357883]]


c:\Users\chris\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:464: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


In [65]:
# import joblib

# # Save the Random Forest model
# joblib.dump(random_forest_model, 'random_forest_model.pkl')

# # Save the scalers
# joblib.dump(scaler_X, 'scaler_X.pkl')
# joblib.dump(scaler_y, 'scaler_y.pkl')


In [66]:
# import tensorflow as tf

# # Load your Keras model
# model = tf.keras.models.load_model('Final_Model')

# # Convert the Keras model to TFLite
# converter = tf.lite.TFLiteConverter.from_keras_model(model)
# tflite_model = converter.convert()

# # Save the TFLite model to a file
# with open('TFLYT\FNN_quant.tflite', 'wb') as f:
#     f.write(tflite_model)


In [67]:
# import pandas as pd
# gg=pd.read_csv("datasets\Genotypic_Data.csv")
# kk=pd.read_csv("pheno_geno")

# dp=[]
# for i in kk.columns:
#     dp.append(i)

# dp=dp[1:]
# dp=dp[:24]

# snps_series = pd.Series(df['SNPs'])
# df=gg
# pos_values = df[df['SNPs'].isin(dp)]['Pos']



# print(pos_values.tolist())
